In [1]:
import numpy as np
import os
import cv2
import keras
import matplotlib.pyplot as plt
from imutils import paths
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [2]:
classes = ['c0', 'c1','c2','c3','c4','c5','c6','c7','c8','c9']
Dict = {'c0' : 0, 'c1' :1, 'c2':2, 'c3':3, 'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
images = []
Img_labels = []
train_path='imgs/train'

In [3]:
for label in classes:
      path = os.path.join(train_path , label)
      for img in os.listdir(path):
         img = cv2.imread(os.path.join(path,img))
         new_img = cv2.resize(img, (64, 64))
         images.append(new_img)
         Img_labels.append(Dict[label])
         # print(images,Img_labels)
     #print(label)

In [4]:
img=np.array(images)
labels=np.array(Img_labels)
print(img.shape)

(22424, 64, 64, 3)


In [5]:
from sklearn.model_selection import train_test_split
x_train,X_test,y_train,Y_test = train_test_split(img,labels, test_size=0.2, random_state=1)
X_train,X_val,Y_train,Y_val = train_test_split(x_train,y_train, test_size=0.1, random_state=1)

In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(X_val.shape)
print(Y_val.shape)


(16145, 64, 64, 3)
(16145,)
(4485, 64, 64, 3)
(4485,)
(1794, 64, 64, 3)
(1794,)


In [8]:
from skimage.feature import local_binary_pattern
def get_lbp(images, name='lbp', save=False):
  result = np.array([local_binary_pattern(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), 10, 3).flatten() for img in images])      
  return result
lbp_train = get_lbp(X_train, name='lbp_train', save=True)
lbp_val = get_lbp(X_val, name='lbp_val', save=True)

In [9]:
print(lbp_train.shape)
print(lbp_val.shape)

(16145, 4096)
(1794, 4096)


In [31]:
from sklearn import preprocessing
def norm_features_minmax(train, test):
      min_max_scaler = preprocessing.MinMaxScaler()
      norm_train = min_max_scaler.fit_transform(train)
      norm_test = min_max_scaler.transform(test)
      return norm_train, norm_test

In [32]:
norm_lbp_train,norm_lbp_val=norm_features_minmax(lbp_train,lbp_val)

In [37]:
print(norm_lbp_train.shape)
print(norm_lbp_val.shape)

(16145, 4096)
(1794, 4096)


In [38]:
from sklearn.decomposition import PCA
pca=PCA(n_components=100)
pca_norm_lbp_train = pca.fit_transform(norm_lbp_train)
pca_norm_lbp_val = pca.transform(norm_lbp_val)

In [39]:
print(pca_norm_lbp_train.shape)
print(pca_norm_lbp_val.shape)

(16145, 100)
(1794, 100)


In [40]:
from sklearn.svm import SVC
model_pca = SVC()
model_pca.fit(pca_norm_lbp_train, Y_train)
accPCA = model_pca.score(pca_norm_lbp_val,Y_val)


In [41]:
print("Accuracy on PCA: ",round(accPCA,5)*100,"%")

Accuracy on PCA:  98.71799999999999 %


In [42]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=9)
lda_norm_lbp_train = lda.fit_transform(norm_lbp_train,Y_train)
lda_norm_lbp_val = lda.transform(norm_lbp_val)

In [43]:
print(lda_norm_lbp_train.shape)
print(lda_norm_lbp_val.shape)

(16145, 9)
(1794, 9)


In [66]:
model_lda = SVC()
model_lda.fit(lda_norm_lbp_train, Y_train)
accLDA=model_lda.score(lda_norm_lbp_val,Y_val)

In [67]:
print("Accuracy on LDA: ",round(accLDA,5)*100,"%")

Accuracy on LDA:  98.774 %


In [47]:
lda = LDA(n_components=9)
lda_o_pca_norm_lbp_train = lda.fit_transform(pca_norm_lbp_train,Y_train)
lda_o_pca_norm_lbp_val = lda.transform(pca_norm_lbp_val)

In [48]:
print(lda_o_pca_norm_lbp_train.shape)
print(lda_o_pca_norm_lbp_val.shape)

(16145, 9)
(1794, 9)


In [49]:
model_lda_o_pca = SVC()
model_lda_o_pca.fit(lda_o_pca_norm_lbp_train, Y_train)
acc_lda_o_pca = model_lda_o_pca.score(lda_o_pca_norm_lbp_val,Y_val)


In [50]:
print("Accuracy on LDA: ",round(acc_lda_o_pca,5)*100,"%")

Accuracy on LDA:  82.441 %


In [53]:
import pickle
with open('model_lda','wb') as f1:
    pickle.dump(model_lda,f1)

with open('model_pca','wb') as f2:
    pickle.dump(model_pca,f2)

with open('mmodel_lda_o_pca','wb') as f3:
    pickle.dump(model_lda_o_pca,f3)

In [54]:
with open('model_lda','rb') as f1:
    model_lda_loaded = pickle.load(f1)

with open('model_pca','rb') as f2:
    model_pca_loaded = pickle.load(f2)

with open('mmodel_lda_o_pca','rb') as f3:
    model_lda_O_pca_loaded = pickle.load(f3)

In [55]:
print(X_test.shape)
print(Y_test.shape)

(4485, 64, 64, 3)
(4485,)


In [56]:
lbp_test = get_lbp(X_test, name='lbp_train', save=True)
print(lbp_test.shape)

(4485, 4096)


In [57]:
norm_lbp_train,norm_lbp_test=norm_features_minmax(lbp_train,lbp_test)

In [58]:
print(norm_lbp_train.shape)
print(norm_lbp_test.shape)

(16145, 4096)
(4485, 4096)


In [59]:
lda = LDA(n_components=9)
lda_norm_lbp_test = lda.fit_transform(norm_lbp_test,Y_test)

In [60]:
print(lda_norm_lbp_test.shape)

(4485, 9)


In [61]:
pca=PCA(n_components=100)
pca_norm_lbp_test = pca.fit_transform(norm_lbp_test)

In [62]:
print(pca_norm_lbp_test.shape)

(4485, 100)


In [63]:
lda = LDA(n_components=9)
lda_o_pca_norm_lbp_test = lda.fit_transform(pca_norm_lbp_test,Y_test)

In [64]:
print(lda_o_pca_norm_lbp_test.shape)

(4485, 9)


In [69]:
accLDA_o_PCA=model_lda_O_pca_loaded.score(lda_o_pca_norm_lbp_test,Y_test)
acc_LDA=model_lda_loaded.score(lda_norm_lbp_test,Y_test)
acc_PCA=model_pca_loaded.score(pca_norm_lbp_test,Y_test)

print("Accuracy on LDA over PCA: ",round(accLDA_o_PCA,10),"%")
print("Accuracy on LDA: ",round(acc_LDA,10),"%")
print("Accuracy on PCA: ",round(acc_PCA,10),"%")

Accuracy on LDA over PCA:  0.4336677815 %
Accuracy on LDA:  0.123522854 %
Accuracy on PCA:  0.1188405797 %
